In [12]:
import tensorflow as tf
import numpy as np

In [14]:
# 5*4*3 with numpy
#layer1=np.array([[1,1,1],[1,0,0],[1,0,0],[1,0,0]])
#layer2=np.array([[0,0,0],[1,0,0],[0,0,0],[0,0,0]])
#layer3=np.array([[0,0,0],[1,1,0],[0,0,0],[0,0,0]])
#layer4=np.array([[0,0,0],[1,0,0],[0,0,0],[0,0,0]])
#layer5=np.array([[0,0,0],[1,0,0],[0,0,0],[0,0,0]])
layer1=np.array([[0,0,1],[0,0,1],[0,0,1],[1,1,1]])
layer2=np.array([[0,0,0],[0,0,0],[0,0,1],[0,0,0]])
layer3=np.array([[0,0,0],[0,0,0],[0,1,1],[0,0,0]])
layer4=np.array([[0,0,0],[0,0,0],[0,0,1],[0,0,0]])
layer5=np.array([[0,0,0],[0,0,0],[0,0,1],[0,0,0]])
logo=np.array([layer1,layer2,layer3,layer4,layer5])

In [16]:
sess=tf.Session()
tflogo=tf.constant(logo)
with sess.as_default():
    print (tflogo.eval())
tflogo.shape

[[[0 0 1]
  [0 0 1]
  [0 0 1]
  [1 1 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 1 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 1]
  [0 0 0]]]


TensorShape([Dimension(5), Dimension(4), Dimension(3)])

In [5]:
# Build the logit model
x=tf.placeholder(tf.float32,[None,6])
ytrue=tf.placeholder(tf.float32,[None,1])
W=tf.Variable(tf.zeros([6,1]))
b=tf.Variable(tf.zeros([1]))
z=tf.matmul(x,W)+b
y=tf.exp(z)/(1+tf.exp(z))
# Negative log likelihood: sum of p^y*1-p^(1-y)
loss=tf.reduce_sum(-ytrue*tf.log(y)-(1-ytrue)*tf.log(1-y))

Instructions for updating:
Colocations handled automatically by placer.


In [117]:
xtest=np.load("logistic_xtest.npy") #500*6
xtrain=np.load("logistic_xtrain.npy") # 2000*6
ytest=np.load("logistic_ytest.npy") # 500*1
ytrain=np.load("logistic_ytrain.npy") # 2000*1

In [118]:
# Train the data
init=tf.global_variables_initializer()
train=tf.train.GradientDescentOptimizer(0.005).minimize(loss)
sess.run(init)
for i in range(1000):
    _,loss_val=sess.run([train,loss],{x:xtrain,ytrue:ytrain})
    #print(loss_val)

In [119]:
print(sess.run([W,b]))

[array([[0.97808135],
       [1.2326875 ],
       [1.4982153 ],
       [3.0153084 ],
       [4.6440253 ],
       [7.528404  ]], dtype=float32), array([-0.95491993], dtype=float32)]


In [120]:
#sess=tf.Session()
# Test the data
# Negative log likelihood
xtest=tf.constant(xtest,dtype=tf.float32)
ytest=tf.constant(ytest,dtype=tf.float32)
z_test=tf.matmul(xtest,W)+b
y_test=tf.exp(z_test)/(1+tf.exp(z_test))
loss_test=tf.reduce_sum(-ytest*tf.log(y_test)-(1-ytest)*tf.log(1-y_test))
# Accuracy
correct=tf.equal(tf.argmax([y_test,1-y_test],0),tf.argmax([ytest,1-ytest],0))
accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [233]:
with sess.as_default():
    print (loss_test.eval())
    print(accuracy.eval())

163.01419
0.858


#### The negative log-likelihood is 163.01419 and the classification accuracy is about 85.8%  

In [122]:
## Calculate the squared error
#sess=tf.Session()
W_real=tf.constant([[1],[1],[2],[3],[5],[8]],dtype=tf.float32)
b_real=tf.constant([-1],dtype=tf.float32)
sq_W=tf.square(W-W_real)
sq_b=tf.square(b-b_real)
with sess.as_default():
    print (sq_W.eval())
    print(sq_b.eval())

[[4.8042741e-04]
 [5.4143459e-02]
 [2.5178787e-01]
 [2.3434650e-04]
 [1.2671797e-01]
 [2.2240256e-01]]
[0.00203221]


#### Mathematically speaking, the squared error is the sum of (bias)^2 and variance, where bias is defined as E(theta_estimation)-theta_true and variance is var(theta_estimation)

In [123]:
results_logistic={"W":sess.run(W),"Wsqerr":sess.run(sq_W),"b":sess.run(b),"bsqerr":sess.run(sq_b),"log_lik_test": sess.run(loss_test)}


In [124]:
results_logistic

{'W': array([[0.97808135],
        [1.2326875 ],
        [1.4982153 ],
        [3.0153084 ],
        [4.6440253 ],
        [7.528404  ]], dtype=float32), 'Wsqerr': array([[4.8042741e-04],
        [5.4143459e-02],
        [2.5178787e-01],
        [2.3434650e-04],
        [1.2671797e-01],
        [2.2240256e-01]], dtype=float32), 'b': array([-0.95491993], dtype=float32), 'bsqerr': array([0.00203221], dtype=float32), 'log_lik_test': 163.01419}

In [66]:
# Classification of normally distributed data with Adagrad
xtest2=np.load("normal_xtest.npy") #500*1
xtrain2=np.load("normal_xtrain.npy") # 2000*1
ytest2=np.load("normal_ytest.npy") # 500*3
ytrain2=np.load("normal_ytrain.npy") # 2000*3

In [70]:
import math as m
x2=tf.placeholder(tf.float32,[None,1])
ytrue2=tf.placeholder(tf.float32,[None,3])
mu=tf.Variable(tf.ones([3]))
sigma2=tf.Variable(tf.ones([3]))
## Loss function -ylog()
# Normal distribution function: 1/sqrt(2*pie*sigma2)*exp(-(x-mu)/2/sigma2)
p=1/tf.sqrt(2*m.pi*sigma2)*tf.exp(-tf.pow((x2-mu),2)/2/sigma2) # 2 probabilities
loss2=tf.reduce_mean(tf.reduce_sum(-ytrue2*tf.log(p),reduction_indices=[1]))
# Add the prediction
prediction=tf.argmax(p,1)+1

In [71]:
sess2=tf.InteractiveSession()
train2=tf.train.AdagradOptimizer(0.08).minimize(loss2)
init2=tf.initialize_all_variables().run()
for i in range(3000):
    _,loss2_val,predict=sess2.run([train2,loss2,prediction],{x2:xtrain2,ytrue2:ytrain2})

C:\Users\lenovo\Envs\pyenv36\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [72]:
with sess2.as_default():
    print (sigma2.eval())
    print (mu.eval())
#print (predict)

[0.5333826 1.0087531 1.5160059]
[-1.0076363   0.00449133  3.0048532 ]


In [73]:
#print (prediction)

In [74]:
# Evaluating on the test data
cross=sess2.run(loss2,{x2:xtest2,ytrue2:ytest2})
print(cross)

1.3726945


In [12]:
## Accuracy 
accuracy_=tf.equal(tf.argmax(p,1),tf.argmax(ytrue2,1))
accuracy2=tf.reduce_mean(tf.cast(accuracy_,tf.float32))
_,errors=sess2.run([accuracy2,1-accuracy2],{x2:xtest2,ytrue2:ytest2})
print(errors)

0.268


####  The cross_entropy of the model on the test data is 686.34717 and  73.2% of the data points are correctly classified (therefore 26.8% are not)

In [13]:
## 
mu_real=tf.constant([-1,0,3],dtype=tf.float32)
sigma2_real=tf.constant([0.5,1,1.5],dtype=tf.float32)
mu_q=tf.square(mu_real-mu)
sigma2_q=tf.square(sigma2_real-sigma2)
with sess2.as_default():
    print (mu_q.eval(),sigma2_q.eval())

[5.8315032e-05 2.0171905e-05 2.3560966e-05] [1.1140475e-03 7.6630691e-05 2.5472092e-04]


#### Mathematically speaking, the squared error is the sum of (bias)^2 and variance, where bias is defined as E(theta_estimation)-theta_true and variance is var(theta_estimation)

In [14]:
results_class={"mu":sess2.run(mu),"sigma2":sess2.run(sigma2),"crossent_test":cross,"class_error":errors}

In [15]:
results_class

{'mu': array([-1.0076364 ,  0.00449131,  3.004854  ], dtype=float32),
 'sigma2': array([0.53337735, 1.0087539 , 1.51596   ], dtype=float32),
 'crossent_test': 686.34717,
 'class_error': 0.268}

In [332]:
# Prediction Node
sess3=tf.Session()
x_pre=tf.placeholder(tf.float32,[None,1])
sigma_p=sigma2.eval()
mu_p=mu.eval()
p=1/tf.sqrt(2*m.pi*sigma_p)*tf.exp(-tf.pow((x_pre-mu_p),2)/2/sigma_p) # 2 probabilities
#loss2=tf.reduce_mean(tf.reduce_sum(-ytrue2*tf.log(p)))
# Add the prediction
prediction=tf.argmax(p,1)+1


In [334]:
# Save the model
# https://cloud.google.com/ml-engine/docs/deploying-models#creating_a_model_version
# https://github.com/GoogleCloudPlatform/cloudml-dist-mnist-example
# https://github.com/qifanoctave/hw10_507
tf.saved_model.simple_save(sess3,"normal_trained",inputs={"x_pre":x_pre},
                           outputs={"prediction":prediction})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.


FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[node save_12/SaveV2 (defined at <ipython-input-334-1e932ab96c54>:6) ]]

Errors may have originated from an input operation.
Input Source operations connected to node save_12/SaveV2:
 Variable_23/Adagrad (defined at <ipython-input-147-c2ffb1fa20b9>:2)	
 Variable_50 (defined at <ipython-input-230-83d09afb4248>:4)	
 Variable_26/Adagrad (defined at <ipython-input-152-c2ffb1fa20b9>:2)	
 Variable_30/Adagrad_1 (defined at <ipython-input-158-721680361aab>:3)	
 Variable_33/Adagrad (defined at <ipython-input-160-721680361aab>:3)	
 Variable_23 (defined at <ipython-input-146-7deac476286d>:6)	
 Variable_29/Adagrad (defined at <ipython-input-155-376ec7f4109b>:2)	
 Variable_39/Adagrad_3 (defined at <ipython-input-172-54755f129bfe>:2)	
 Variable_41/Adagrad_4 (defined at <ipython-input-183-350b09322000>:2)	
 Variable_47/Adagrad_1 (defined at <ipython-input-220-c0dc77c58359>:2)	
 Variable_53 (defined at <ipython-input-247-b9641c

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`